<a href="https://colab.research.google.com/github/Sharaborina/ChatBot/blob/main/Danish_text_generator%20(based%20on%20TRAX).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

As you know, I am going to develop a chatbot for hotel searching. A user send me in telegram some replica and the next steps are:

- a function tries to extract all valuable data (dates and cities) and save it in a dictionary
- a bot will ask until the dictionary is fill
- if the dictionary is full, then a booking API is run and returns links to appropriate hotels.
- then after we run generative model to answer to his words.


In this Colab file I have trained a dialogue generator model using Trax library. 

Trax is an end-to-end library for deep learning that focuses on clear code and speed. It is actively used and maintained in the Google Brain team.


# **Part 1: Exploring the MultiWoz dataset**

The training is based on the MultiWoz dataset. This dataset has more than 10,000 human annotated dialogues and spans multiple domains and topics. Some dialogues include multiple domains and others include single domains. Also, there is an additional difficulty the dataset MultiWoz is in Englsh, therefore to train on a Danish model, it was decided to translate dialogues to Danish using `GoogleTranslate` API.

In this section, the dataset will be loaded and explored, as well as develop a function to extract the dialogues. 

Let’s first install and import the modules we will be using:

In [1]:
# !git clone 'https://github.com/shsarv/ChatBot.git'
# !mv ChatBot ./drive/MyDrive/ColabNotebooks/

#download a vocabular
# reader = open('/content/drive/MyDrive/ColabNotebooks/corpus/en_50k.subword', mode='r')
# kk = 0
# output = []
# for line in reader:
#   kk += 1
#   output += line.split('	')[0:-1]
# print(kk)
# reader.close()

# reader = open('/content/drive/MyDrive/ColabNotebooks/corpus/en_50k_pruned.subword', mode='w', encoding='utf-8')
# reader.write('\n'.join(output))
# reader.close()
# 
# # Install JAX.
!pip install --upgrade jax --quiet
!pip install --upgrade jaxlib --quiet
!pip install --upgrade trax --quiet
!pip install trax deep_translator tensor2tensor --quiet

# Make sure the Colab Runtime is set to Accelerator: TPU.
import requests
import os
 
import json
import random
import numpy as np
from termcolor import colored
import trax   
from trax import layers as tl
from trax.supervised import training
import tensor2tensor
from tensor2tensor.data_generators import generator_utils
from deep_translator import GoogleTranslator

trax.fastmath.set_backend('jax') #or tensorflow-numpy

# if 'TPU_DRIVER_MODE' not in globals():
#   url = 'http://' + os.environ['COLAB_TPU_ADDR'].split(':')[0] + ':8475/requestversion/tpu_driver0.1-dev20191206'
#   resp = requests.post(url)
#   TPU_DRIVER_MODE = 1

# The following is required to use TPU Driver as JAX's backend.
from jax.config import config
# config.FLAGS.jax_xla_backend = "tpu_driver"
# config.FLAGS.jax_backend_target = "grpc://" + os.environ['COLAB_TPU_ADDR']
print(config.FLAGS.jax_backend_target)







     |████████████████████████████████| 45.3MB 146kB/s 
     |████████████████████████████████| 634kB 7.1MB/s 
     |████████████████████████████████| 4.3MB 10.0MB/s 
     |████████████████████████████████| 153kB 50.9MB/s 
     |████████████████████████████████| 256kB 54.7MB/s 
     |████████████████████████████████| 61kB 7.7MB/s 
     |████████████████████████████████| 1.2MB 58.3MB/s 
     |████████████████████████████████| 2.3MB 56.5MB/s 
     |████████████████████████████████| 368kB 56.2MB/s 
     |████████████████████████████████| 3.9MB 56.2MB/s 
     |████████████████████████████████| 901kB 59.2MB/s 
     |████████████████████████████████| 3.3MB 56.1MB/s 
     |████████████████████████████████| 1.5MB 8.3MB/s 
     |████████████████████████████████| 983kB 56.4MB/s 
     |████████████████████████████████| 686kB 36.1MB/s 
     |████████████████████████████████| 655kB 60.0MB/s 
     |████████████████████████████████| 368kB 52.4MB/s 
     |████████████████████████████████| 81kB 9.4MB/s

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 1.1 Define all global variables as filenames and their paths, mode of run, number of training iterations and so on.

In [12]:
# filename of the MultiWOZ dialogue dataset
ENGLISH_DATA_FILE = 'data.json'
DANISH_DATA_FILE = 'danish_data.json'
# data directory
DATA_DIR = '/content/drive/MyDrive/ColabNotebooks/MultiWoz/'

# loading of already translated array of texts which is kept inside a corresponding file.
LOAD_UNTOKENIZED_DATA_DANISH = True
UNTOKENIZED_DATA_DANISH_FILE = 'untokenized_data_danish.txt'

#creating of a subword dictionary using UNTOKENIZED_DATA_DANISH_FILE or load from file
CREATE_AND_SAVE_SUBWORD_VOCAB = False
# vocabulary filename
VOCAB_FILE = 'en_50k.subword'
# vocabulary file directory
VOCAB_DIR = '/content/drive/MyDrive/ColabNotebooks/MultiWoz/'
# the size of a subword vocabulary
VOCAB_SIZE = 50000

MODEL_DIR = '/content/drive/MyDrive/ColabNotebooks/Generative_model'

N_LAYERS = 6
TRAIN_STEPS = 1000
LOAD_MODEL = False
TRAIN = True

In [13]:
# Let's open the dataset README to see how the dialogues are stored
# with open(DATA_DIR + '/README') as file:
#     print(file.read())

##1.2 Load the MultiWoz dataset

In [14]:
# dictionary where we will load the dialogue dataset
DIALOGUE_ENG_DB = {}
DIALOGUE_DB = {}
# help function to load a JSON file
def load_json(directory, file):
    with open(f'{directory}/{file}') as f: 
        db = json.load(f)
    return db

def upload_json(directory, file, db):
    with open(f'{directory}/{file}', mode='w') as f: 
        json.dump(db, f)

# load the dialogue data set into our dictionary
DIALOGUE_ENG_DB = load_json(DATA_DIR, ENGLISH_DATA_FILE)

#1.3 Exploring of dataset, investigation of its structure

In [15]:
print(f'The number of dialogues is: {len(DIALOGUE_ENG_DB)}')
print('The list of different dialogues', list(DIALOGUE_ENG_DB.keys())[0:7])

The number of dialogues is: 10438
The list of different dialogues ['SNG01856.json', 'SNG0129.json', 'PMUL1635.json', 'MUL2168.json', 'SNG0073.json', 'SNG01445.json', 'MUL2105.json']


In [16]:
# Let's see what keys are stored in the first file above
file = 'SNG01856.json'
print(DIALOGUE_ENG_DB[file].keys())

dict_keys(['goal', 'log'])


In [17]:
# 'goal' keeps the metadata of the conversation
DIALOGUE_ENG_DB[file]['goal'] 

{'attraction': {},
 'hospital': {},
 'hotel': {'book': {'day': 'tuesday',
   'invalid': False,
   'people': '6',
   'pre_invalid': True,
   'stay': '2'},
  'fail_book': {'stay': '3'},
  'fail_info': {},
  'info': {'internet': 'yes',
   'parking': 'yes',
   'pricerange': 'cheap',
   'type': 'hotel'}},
 'message': ["You are looking for a <span class='emphasis'>place to stay</span>. The hotel should be in the <span class='emphasis'>cheap</span> price range and should be in the type of <span class='emphasis'>hotel</span>",
  "The hotel should <span class='emphasis'>include free parking</span> and should <span class='emphasis'>include free wifi</span>",
  "Once you find the <span class='emphasis'>hotel</span> you want to book it for <span class='emphasis'>6 people</span> and <span class='emphasis'>3 nights</span> starting from <span class='emphasis'>tuesday</span>",
  "If the booking fails how about <span class='emphasis'>2 nights</span>",
  "Make sure you get the <span class='emphasis'>ref

In [18]:
# get first element of the log list which corresponds to one replica of a dialogue.
DIALOGUE_ENG_DB[file]['log'][0]

{'dialog_act': {'Hotel-Inform': [['Type', 'hotel'], ['Price', 'cheap']]},
 'metadata': {},
 'span_info': [['Hotel-Inform', 'Type', 'hotel', 20, 20],
  ['Hotel-Inform', 'Price', 'cheap', 10, 10]],
 'text': 'am looking for a place to to stay that has cheap price range it should be in a type of hotel'}

In [19]:
# Let's see the first 2 replicas of a dialogue
print(' Person 1: ', DIALOGUE_ENG_DB[file]['log'][0]['text'])
print(' Person 2: ', DIALOGUE_ENG_DB[file]['log'][1]['text'])

 Person 1:  am looking for a place to to stay that has cheap price range it should be in a type of hotel
 Person 2:  Okay, do you have a specific area you want to stay in?


In [20]:
# this function helps us to save conversation to a long string with Person 1 and Person 2 delimiters.
def get_conversation(file, data_db):
    '''
    Args:
        file (string): filename of the dialogue file saved as json
        data_db (dict): dialogue database
    
    Returns:
        string: A string containing the 'text' fields of  data[file]['log'][x]
    '''
    
    # initialize empty string
    result = ''
    
    # get length of file's log list
    len_msg_log = len(data_db[file]['log'])
    
    # set the delimiter strings
    delimiter_1 = ' Person 1: '
    delimiter_2 = ' Person 2: '
    
    # loop over the file's log list
    for i in range(len_msg_log):
        
    ### START CODE HERE (REPLACE INSTANCES OF 'None' WITH YOUR CODE) ###
    
        # get i'th element of file log list
        cur_log = data_db[file]['log'][i]['text'].replace('\n', ' ').replace('\r', ' ')
        
        # check if i is even
        if i%2 == 0:                   
            # append the 1st delimiter string
            result += delimiter_1
        else: 
            # append the 2nd delimiter string
            result += delimiter_2
        
        # append the message text from the log
        result += cur_log
    
    ### END CODE HERE ###

    return result

In [21]:
# #Translate Database:
# 
# def translate_db(data_db):
#   for file, val in data_db.items():
#     for i in range(len(data_db[file]['log'])):
#       src = data_db[file]['log'][i]['text']
#       # print('src=', src)
#       target = GoogleTranslator(source='auto', target='da').translate(src)
#       data_db[file]['log'][i]['text'] = target
#       # print(target)
#   return data_db

# DIALOGUE_DB = translate_db(DIALOGUE_ENG_DB)

# upload_json(DATA_DIR, DANISH_DATA_FILE, DIALOGUE_DB)
DIALOGUE_DB = DIALOGUE_ENG_DB

In [22]:
conversation = get_conversation(file, DIALOGUE_DB)

In [23]:
# this function beautifully prints a conversation between two people.
def print_conversation(conversation):
    
    if type(conversation) == list:
      for i,c in enumerate(conversation):
        print("Dialogue #{}".format(i+1))
        print_conversation(c)
        print("\n")
    else:
      delimiter_1 = 'Person 1: '
      delimiter_2 = 'Person 2: '
      
      split_list_d1 = conversation.split(delimiter_1)
      
      for sublist in split_list_d1[1:]:
          split_list_d2 = sublist.split(delimiter_2)
          print(colored(f'Person 1: {split_list_d2[0]}', 'red'))
          
          if len(split_list_d2) > 1:
              print(colored(f'Person 2: {split_list_d2[1]}', 'green'))

            
print_conversation(conversation)

Person 1: am looking for a place to to stay that has cheap price range it should be in a type of hotel 
Person 2: Okay, do you have a specific area you want to stay in? 
Person 1: no, i just need to make sure it's cheap. oh, and i need parking 
Person 2: I found 1 cheap hotel for you that includes parking. Do you like me to book it? 
Person 1: Yes, please. 6 people 3 nights starting on tuesday. 
Person 2: I am sorry but I wasn't able to book that for you for Tuesday. Is there another day you would like to stay or perhaps a shorter stay? 
Person 1: how about only 2 nights. 
Person 2: Booking was successful. Reference number is : 7GAWK763. Anything else I can do for you? 
Person 1: No, that will be all. Good bye. 
Person 2: Thank you for using our services.


# **Part 2: Preprocessing and translation**
In this section, I prepare the data for training, namely put the whole conversation to the `untokenized_data` list. Thus, each element of this list corresponds to one dialogue. If you don't have this list, then it can be created (`LOAD_UNTOKENIZED_DANISH` should be `False`), otherwise it is allowed to load from the `DATA_DIR + UNTOKENIZED_DATA_DANISH_FILE` (`LOAD_UNTOKENIZED_DANISH` should be `True`). 

In [24]:
# Translate text
def translate(text, target='da'):
  if type(text) == list:
    out = []
    for t in text:
      print(t)
      out.append(GoogleTranslator(source='auto', target=target).translate(t))
    return out
  return GoogleTranslator(source='auto', target=target).translate(text)


if not LOAD_UNTOKENIZED_DATA_DANISH:
  # the keys are the file names
  all_files = DIALOGUE_DB.keys()

  # initialize empty list
  untokenized_data = []

  # loop over all files
  for file in all_files:
      # this is the graded function you coded
      # returns a string delimited by Person 1 and Person 2
      result = get_conversation(file, DIALOGUE_DB)
      translated = translate(result, target='da')
      # append to the list
      untokenized_data.append(translated)

  #save the processed file
  upload_json(DATA_DIR, UNTOKENIZED_DATA_DANISH_FILE, untokenized_data)
  # print the first two dialogues to check if it's the same as the one we got before
  print_conversation(untokenized_data[0:2])
else: # read the untokenized_data list from a file
  untokenized_data = load_json(DATA_DIR, UNTOKENIZED_DATA_DANISH_FILE)
  print_conversation(untokenized_data[0:2])

Dialogue #1
Person 1: er på udkig efter et sted at bo, der har en billig prisinterval, det skal være i en type hotel 
Person 2: Okay, har du et bestemt område, du vil bo i? 
Person 1: nej, jeg skal bare sørge for, at det er billigt. åh, og jeg har brug for parkering 
Person 2: Jeg fandt et billigt hotel til dig, der inkluderer parkering. Kan du lide, at jeg bestiller det? 
Person 1: Ja tak. 6 personer 3 nætter startende på tirsdag. 
Person 2: Jeg er ked af det, men jeg kunne ikke booke det til dig til tirsdag. Er der en anden dag, du gerne vil bo, eller måske et kortere ophold? 
Person 1: hvad med kun 2 nætter. 
Person 2: Booking lykkedes. Reference nummer er: 7GAWK763. Noget andet jeg kan gøre for dig? 
Person 1: Nej, det er alt. Farvel. 
Person 2: Tak fordi du bruger vores tjenester.


Dialogue #2
Person 1: Hej, jeg er blevet røvet. Kan du hjælpe mig med at komme i kontakt med politiet? 
Person 2: Parkside Police Station er i Parkside, Cambridge. Deres nummer er 01223358966. Er der n

## 2.1 Creating of subword vocabulary
The experiments in the field of NLP show us that training on subwords works better than training on words. Therefore here we create a vocabulary of subwords and save it in `VOCAB_DIR/VOCAB_FILE`using `get_or_generate_txt_vocab` function.

In [25]:
if CREATE_AND_SAVE_SUBWORD_VOCAB:
  !rm -rf $VOCAB_DIR/$VOCAB_FILE
  generator_utils.get_or_generate_txt_vocab(VOCAB_DIR, VOCAB_FILE, VOCAB_SIZE, DATA_DIR + UNTOKENIZED_DATA_DANISH_FILE)
  print("The vocab is succesfully created and saved as {}/{}".format(VOCAB_DIR, VOCAB_FILE))

## 2.2 Splitting into train and test data
To develop a sustainable and reliable model, let us split the list to a train and eval dataset.



In [27]:
# shuffle the list we generated above
random.shuffle(untokenized_data)

# define a cutoff (5% of the total length for this assignment)
# convert to int because we will use it as a list index
cut_off = int(len(untokenized_data) * .1)

# slice the list. the last elements after the cut_off value will be the eval set. the rest is for training. 
train_data, eval_data = untokenized_data[:-cut_off], untokenized_data[-cut_off:]

print(f'number of conversations in the data set: {len(untokenized_data)}')
print(f'number of conversations in train set: {len(train_data)}')
print(f'number of conversations in eval set: {len(eval_data)}')

number of conversations in the data set: 10438
number of conversations in train set: 9395
number of conversations in eval set: 1043


# **2.1 Tokenizing, batching with bucketing**
We can now proceed in generating tokenized batches of our data. Let’s first define a utility generator function to yield elements from our datasets:

In [28]:
def stream(data):
    # loop over the entire data
    while True:
        # get a random element
        d = random.choice(data)
        # yield a tuple pair of identical values 
        # (i.e. our inputs to the model will also be our targets during training)
        yield (d, d)

Now let’s define our data pipeline for tokenizing and batching our data. As in the previous assignments, we will bucket by length and also have an upper bound on the token length.

In [29]:
# trax allows us to use combinators to generate our data pipeline
data_pipeline = trax.data.Serial(
    # randomizes to avoid bias in the way data is displayed
    trax.data.Shuffle(),
    
    #Tokenizes using a subword dict, it breaks words into common prefixes and suffixes for better generalization
    trax.data.Tokenize(vocab_dir=VOCAB_DIR,
                       vocab_file=VOCAB_FILE),
    
    # filter too long sequences
    trax.data.FilterByLength(1024),
    
    # bucket by length
    trax.data.BucketByLength(boundaries=[128, 256,  512, 1024],
                             batch_sizes=[16, 8,    4,   2, 1]), #BE CAREFUL!
                              #  batch_sizes=[128, 64,    32,   16, 8]),
                                # batch_sizes=[32, 16,    8,   4, 2]), #BE CAREFUL!
    # add loss weights but do not add it to the padding tokens (i.e. 0)
    trax.data.AddLossWeights(id_to_mask=0)
)

# apply the data pipeline to our train and eval sets
train_stream = data_pipeline(stream(train_data))
eval_stream = data_pipeline(stream(eval_data))

Peek into the train stream.

In [30]:
# the stream generators will yield (input, target, weights). let's just grab the input for inspection
inp, _, _ = next(train_stream)

# print the shape. format is (batch size, token length)
print("input shape: ", inp.shape)

# detokenize the first element

print(trax.data.detokenize(inp[0], vocab_dir=VOCAB_DIR, vocab_file=VOCAB_FILE))


input shape:  (2, 1024)
Person 1: Jeg har brug for et hotelværelse i nærheden af ​​Cambridge på fredag. Kan du hjælpe? Person 2: Du valgte attraktion, og hotellerne kommer ikke op. Jeg kan hjælpe med attraktioner. Person 1: Jeg vil gerne have en anbefaling til en attraktion at besøge i centret tak. Person 2: Det afhænger af, hvad du kan lide - kan du lide museer? Kollegier? Musik? Lad mig vide, så jeg kan anbefale noget Person 1: Jeg er ikke særlig. Jeg leder bare efter noget at gøre. Er der noget, som du kan anbefale som en, du ikke vil gå glip af, mens du er i byen? Person 2: Vi har en fantastisk biograf her i byen. Det kaldes Vue Cinema. Det er i centrum af byen. Ville det lyde som noget, du ville være interesseret i? Person 1: Ja, kan jeg få adresse og telefonnummer? Person 2: Sikker! De er placeret på The Grafton Center, East Road, og deres telefonnummer er 08712240240. Kan jeg hjælpe dig med noget andet? Person 1: Jeg er også interesseret i at finde et tog til Cambridge fra Norwi

# Part 3: ReformerLM Training
Let's use the pre-built model which is already implemented in Trax.
Below a wrapper function is implemented that returns a Reformer Language Model.

In [35]:
def ReformerLM(vocab_size=VOCAB_SIZE, n_layers=2, mode='train', attention_type=tl.SelfAttention):
    """
    Args: 
        vocab_size (int): size of the vocabulary
        n_layers (int): number of decoder layers
        mode (string): setting of the model which can be 'train', 'eval', or 'predict' 
        attention_type(class): attention class to use 
    Returns: 
        model (ReformerLM): a reformer language model implemented in Trax
    """    
    
    ### START CODE HERE (REPLACE INSTANCES OF 'None' WITH YOUR CODE) ###
    # initialize an instance of Trax's ReformerLM class
    model = trax.models.reformer.ReformerLM( 
        # set vocab size
        vocab_size = vocab_size,
        # set number of layers
        n_layers = n_layers,
        # set mode
        mode = mode,
        # set attention type
        attention_type = attention_type
    )
    
    ### END CODE HERE ###
    return model

# **Training loop**
Here a function `training_loop` that takes in my model and trains it.

In [36]:
def training_loop(ReformerLM, train_gen, eval_gen, n_layers=2, output_dir = MODEL_DIR):
    lr_schedule = trax.lr.warmup_and_rsqrt_decay(n_warmup_steps=1000, max_value=0.008)
    
    train_task = training.TrainTask(
        labeled_data=train_gen,
        loss_layer=tl.WeightedCategoryCrossEntropy(),
        optimizer=trax.optimizers.Adam(0.01),
        lr_schedule=lr_schedule,
        n_steps_per_checkpoint=5
    )
    
    eval_task = training.EvalTask(
        labeled_data=eval_gen,
        metrics=[tl.WeightedCategoryCrossEntropy(), tl.WeightedCategoryAccuracy()]
    )
    
    loop = training.Loop(model=ReformerLM(n_layers=n_layers),
                         tasks=[train_task],
                         eval_tasks=[eval_task],
                         output_dir=output_dir)
    
    return loop

In [37]:
# training of a model or its loading from the file MODEL_DIR + '/model.pkl.gz'

if LOAD_MODEL == False:
  print("Creating a new model...")
  !rm -f $MODEL_DIR/model.pkl.gz
  loop = training_loop(ReformerLM, train_stream, eval_stream, n_layers=N_LAYERS)
else:
  print("Loading a new model")
  loop = training_loop(ReformerLM, train_stream, eval_stream, n_layers=N_LAYERS)
  loop.model.init_from_file(MODEL_DIR + '/model.pkl.gz')

if TRAIN == True:
  loop.run(TRAIN_STEPS)

Creating a new model...


/usr/local/lib/python3.7/dist-packages/jax/lib/xla_bridge.py:304: UserWarning: jax.host_id has been renamed to jax.process_index. This alias will eventually be removed; please update your code.
  "jax.host_id has been renamed to jax.process_index. This alias "
/usr/local/lib/python3.7/dist-packages/jax/lib/xla_bridge.py:317: UserWarning: jax.host_count has been renamed to jax.process_count. This alias will eventually be removed; please update your code.
  "jax.host_count has been renamed to jax.process_count. This alias "



Step      1: Total number of trainable weights: 96802640
Step      1: Ran 1 train steps in 116.68 secs
Step      1: train WeightedCategoryCrossEntropy |  10.84641933
Step      1: eval  WeightedCategoryCrossEntropy |  10.84339046
Step      1: eval      WeightedCategoryAccuracy |  0.00000000

Step      5: Ran 4 train steps in 126.67 secs
Step      5: train WeightedCategoryCrossEntropy |  10.78242779
Step      5: eval  WeightedCategoryCrossEntropy |  10.62368202
Step      5: eval      WeightedCategoryAccuracy |  0.01458080

Step     10: Ran 5 train steps in 143.59 secs
Step     10: train WeightedCategoryCrossEntropy |  10.46451664
Step     10: eval  WeightedCategoryCrossEntropy |  10.21728039
Step     10: eval      WeightedCategoryAccuracy |  0.05366269

Step     15: Ran 5 train steps in 70.65 secs
Step     15: train WeightedCategoryCrossEntropy |  9.98524952
Step     15: eval  WeightedCategoryCrossEntropy |  9.57501507
Step     15: eval      WeightedCategoryAccuracy |  0.04920914

Step 

In [ ]:
loop.run(TRAIN_STEPS)

/usr/local/lib/python3.7/dist-packages/jax/lib/xla_bridge.py:304: UserWarning: jax.host_id has been renamed to jax.process_index. This alias will eventually be removed; please update your code.
  "jax.host_id has been renamed to jax.process_index. This alias "



Step   1005: Ran 5 train steps in 71.76 secs
Step   1005: train WeightedCategoryCrossEntropy |  3.44074559
Step   1005: eval  WeightedCategoryCrossEntropy |  3.34850526
Step   1005: eval      WeightedCategoryAccuracy |  0.32222223

Step   1010: Ran 5 train steps in 70.59 secs
Step   1010: train WeightedCategoryCrossEntropy |  3.50347400
Step   1010: eval  WeightedCategoryCrossEntropy |  3.37727499
Step   1010: eval      WeightedCategoryAccuracy |  0.31296450

Step   1015: Ran 5 train steps in 70.47 secs
Step   1015: train WeightedCategoryCrossEntropy |  3.43329811
Step   1015: eval  WeightedCategoryCrossEntropy |  3.38387275
Step   1015: eval      WeightedCategoryAccuracy |  0.32614380

Step   1020: Ran 5 train steps in 72.20 secs
Step   1020: train WeightedCategoryCrossEntropy |  3.47485161
Step   1020: eval  WeightedCategoryCrossEntropy |  3.41472888
Step   1020: eval      WeightedCategoryAccuracy |  0.33101448

Step   1025: Ran 5 train steps in 70.23 secs
Step   1025: train Weighte

# Part 5:   Decode from a pretrained model

The decoding is proceeded on using the model architecture just implemented. The [autoregressive_sample_stream()](https://trax-ml.readthedocs.io/en/latest/trax.supervised.html#trax.supervised.decoding.autoregressive_sample_stream) decoding method from Trax is used to do fast inference. Let's define a few parameters to initialize our model.



In [ ]:
# define the `predict_mem_len` and `predict_drop_len` of tl.SelfAttention
def attention(*args, **kwargs):
    # number of input positions to remember in a cache when doing fast inference. 
    kwargs['predict_mem_len'] = 128
    # number of input elements to drop once the fast inference input cache fills up.
    kwargs['predict_drop_len'] = 128
    # return the attention layer with the parameters defined above
    return tl.SelfAttention(*args, **kwargs)

# define the model using the ReformerLM function you implemented earlier.
model = ReformerLM(
    vocab_size=VOCAB_SIZE,
    n_layers=N_LAYERS,
    mode='predict',
    attention_type=attention,
)

# TRAX needs the model to be initialized with this shape
# define an input signature so we can initialize our model. shape will be (1, 1) and the data type is int32.
shape11 = trax.shapes.ShapeDtype((1, 1), dtype=np.int32)
model.init(shape11)

# Loading weights from the trained model
model.weights = loop.eval_model.weights

# saving the starting state for each new dialogue prediction
STARTING_STATE = model.state

In [ ]:
str(model) == str(loop.eval_model)

In [ ]:
def tokenize(sentence, vocab_file, vocab_dir):
    return list(trax.data.tokenize(iter([sentence]), vocab_file=vocab_file, vocab_dir=vocab_dir))[0]

def detokenize(tokens, vocab_file, vocab_dir):
    return trax.data.detokenize(tokens, vocab_file=vocab_file, vocab_dir=vocab_dir)


In [ ]:
# # initialize from file
# model.init_from_file('chatbot_model1.pkl.gz',
#                      weights_only=True, input_signature=shape11)

# # save the starting state
# STARTING_STATE = model.state

In [ ]:
def ReformerLM_output_gen(ReformerLM, start_sentence, vocab_file, vocab_dir, temperature):
    """
    Args:
        ReformerLM:  the Reformer language model you just trained
        start_sentence (string): starting sentence of the conversation
        vocab_file (string): vocabulary filename
        vocab_dir (string): directory of the vocabulary file
        temperature (float): parameter for sampling ranging from 0.0 to 1.0.
            0.0: same as argmax, always pick the most probable token
            1.0: sampling from the distribution (can sometimes say random things)

    Returns:
        generator: yields the next symbol generated by the model
    """
    
    # Create input tokens using the the tokenize function
    input_tokens = tokenize(start_sentence, vocab_file, vocab_dir)
    
    # Add batch dimension to array. Convert from (n,) to (x, n) where 
    # x is the batch size. Default is 1. (hint: you can use np.expand_dims() with axis=0)
    input_tokens_with_batch = input_tokens[None]
    
    # call the autoregressive_sample_stream function from trax
    output_gen = trax.supervised.decoding.autoregressive_sample_stream( 
        # model
        model = ReformerLM,
        # inputs will be the tokens with batch dimension
        inputs = input_tokens_with_batch,
        # temperature
        temperature = temperature
    )
    return output_gen

In [ ]:
def generate_dialogue(ReformerLM, model_state, start_sentence, vocab_file, vocab_dir, max_len, temperature):
    delimiter_1 = 'Person 1: ' 
    delimiter_2 = 'Person 2: '
    sentence = ''
    counter = 0
    
    result = [tokenize(': ', vocab_file=vocab_file, vocab_dir=vocab_dir)]
    
    ReformerLM.state = model_state
    
    output = ReformerLM_output_gen(ReformerLM, start_sentence, vocab_file=VOCAB_FILE, vocab_dir=VOCAB_DIR, temperature=temperature)
    
    print(colored(start_sentence.split(delimiter_2)[0].strip(), 'green'))
    
    for o in output:
        result.append(o)
        
        sentence = detokenize(np.concatenate(result, axis=0), vocab_file=VOCAB_FILE, vocab_dir=VOCAB_DIR)
        # print(sentence)
        if sentence.endswith(delimiter_1):
            sentence = sentence.split(delimiter_1)[0]
            print(colored(f'{delimiter_2}{sentence}', 'red'))
            sentence = ''
            result.clear()
        
        elif sentence.endswith(delimiter_2):
            sentence = sentence.split(delimiter_2)[0]
            print(colored(f'{delimiter_1}{sentence}', 'green'))
            sentence = ''
            result.clear()

        counter += 1
        
        if counter > max_len:
            print(sentence)
            break

In [ ]:
sample_sentence = ' Person 1: Er der et hospital noget sted? Person 2: '
generate_dialogue(ReformerLM=model, model_state=STARTING_STATE, start_sentence=sample_sentence, vocab_file=VOCAB_FILE, vocab_dir=VOCAB_DIR, max_len=200, temperature=0.1)

In [44]:
sample_sentence = ' Person 1: Er der et teater i byen? Person 2: '
generate_dialogue(ReformerLM=model, model_state=STARTING_STATE, start_sentence=sample_sentence, vocab_file=VOCAB_FILE, vocab_dir=VOCAB_DIR, max_len=200, temperature=0.1)


Person 1: Er der et teater i byen?
Person 2: : Jeg ji centrum byen. Jeg vil du du h2230;rrr〓2230;rcentrum 
Person 2: Jeg for at du hfor du du du du du have. 
Jeg har du du du du du du du du du du du du du du du du du du du du du du du du du vil du du du du du du du du du du du du du hjࢵr〓;;;;230;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;2230;;30;;;;;30;230;;;;;;;;;;;;;;;;230;;;;


In [46]:
sample_sentence = ' Person 1: Kan du reservere en taxa? Person 2: '
generate_dialogue(ReformerLM=model, model_state=STARTING_STATE, start_sentence=sample_sentence, vocab_file=VOCAB_FILE, vocab_dir=VOCAB_DIR, max_len=200, temperature=0.1)

Person 1: Kan du reservere en taxa?
Person 2: : Jeg har brug for at du finde at du finde at finde et et sted? 
Person 1: Jeg vil du du finde at du du du du du du du du hj〓r
Person 2: Jeg vil du du have? 
Person 2: Jeg har efter mig. Jeg har brug for at du du have? 
Person 2: Jeg vil du du har brug for? 
Jeg vil du du du du du hjࢶrࢵ230;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;30;230;;;;;230;;;;;;;;;;;;;;;;;;;;;230;;;
